In [ ]:
from requests import get
from bs4 import BeautifulSoup
from time import time, sleep
from random import randint
from IPython.core.display import clear_output
headers = {"Accept-Language": "en-US, en;q=0.5"}

# Redeclare the lists to store data in
names = []
years = []
imdb_ratings = []
metascores = []
directors = []
votes = []
gross = []

# Prepare the loop
start_time = time()
request = 0

pages = [str(i) for i in range(1,5)]
years_url = [str(i) for i in range(2010,2020)]

# For every year in the interval 2010-2019
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        # Exp: https://www.imdb.com/search/title/?release_date=2019&sort=num_votes,desc&page=1
        response = get('http://www.imdb.com/search/title?release_date='+year_url+'&sort=num_votes,desc&page='+page, headers = headers)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        request += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(request, response.status_code))

        # Break the loop if the number of requests is greater than expected
        # 4 pages * 10 years = 40 requests
        if request > 40:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        mv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for container in mv_containers:
            # If the movie has a Metascore, then:
            if container.find('div', class_ = 'ratings-metascore') is not None:

                # Scrape the name
                name = container.h3.a.text
                names.append(name)

                # Scrape the year
                year = container.h3.find('span', class_ = 'lister-item-year').text
                years.append(year)

                # Scrape the IMDB rating
                imdb = float(container.strong.text)
                imdb_ratings.append(imdb)

                # Scrape the Metascore
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))
                
                # Scrape the directors
                director = ''.join(container.find('p', class_ = '').text.split('Stars')[0].split('\n')[2:-2])
                directors.append(director)

                # Scrape the number of votes
                vote = container.find_all('span', attrs = {'name':'nv'})[0]['data-value']
                votes.append(int(vote))
                
                # If the movie has a Gross, then:
                if len(container.find_all('span', attrs = {'name':'nv'})) >= 2:
                    
                    # Scrape the gross
                    gross_value = container.find_all('span', attrs = {'name':'nv'})[1]['data-value']
                    gross.append(gross_value)
                    
                else:
                    gross.append("-")        